Some basic structure below to get started.

Fetching on-chain Data from Dune API

In [ ]:
import requests
import os
from dotenv import load_dotenv
from dune_client.client import DuneClient
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase


# Clear any existing environment variables first
if 'DUNE_API_KEY' in os.environ:
    del os.environ['DUNE_API_KEY']

# Load environment variables from .env file in project root
load_dotenv('../.env', override=True)

API_KEY = os.getenv('DUNE_API_KEY')      # Make sure your .env file has DUNE_API_KEY=your_actual_api_key
# print(f"API_KEY: {API_KEY  }")

if API_KEY:
    dune = DuneClient(API_KEY)
    df = dune.get_latest_result_dataframe(5745512)  # From https://dune.com/troutmax/onchain-metrics dashboard
    print("Query executed successfully!")
    print("===============================================================================")
    print(df.head())
    
else:
    print("API key not found - please check your .env file")

Query executed successfully!
                    block_date blockchain               volumeUSD  \
0  2023-09-16 00:00:00.000 UTC   Ethereum  1.3475834982058318e+07   
1  2023-09-16 00:00:00.000 UTC        BSC      105265.85424866767   
2  2023-09-16 00:00:00.000 UTC   Arbitrum       668.8733134123446   
3  2023-09-16 00:00:00.000 UTC     Solana           292.982798699   
4  2023-09-17 00:00:00.000 UTC   Ethereum  1.1929342885034062e+07   

        botRevenueUSD  numberOfUsers  numberOfNewUsers  numberOfTrades  \
0  36986.727368199536           8322              8322         50526.0   
1               <nil>           1193              1193          5102.0   
2               <nil>              3                 3             3.0   
3       1.75426410688              1                 1            11.0   
4    37326.3504461101           7526              3014         44517.0   

  averageVolumePerUserUSD averageVolumePerTradeUSD  
0      1619.3024491778801        266.7109009630352  
1    

In [108]:
df.info()
print("=====================================================================================")
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4591 entries, 0 to 4590
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   block_date                4591 non-null   datetime64[ns, UTC]
 1   blockchain                4591 non-null   object             
 2   volumeUSD                 4591 non-null   object             
 3   botRevenueUSD             4591 non-null   object             
 4   numberOfUsers             4591 non-null   int64              
 5   numberOfNewUsers          4591 non-null   int64              
 6   numberOfTrades            4591 non-null   float64            
 7   averageVolumePerUserUSD   4591 non-null   object             
 8   averageVolumePerTradeUSD  4591 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(1), int64(2), object(5)
memory usage: 322.9+ KB


,block_date,blockchain,volumeUSD,botRevenueUSD,numberOfUsers,numberOfNewUsers,numberOfTrades,averageVolumePerUserUSD,averageVolumePerTradeUSD
0,2023-09-16 00:00:00+00:00,Ethereum,1.3475834982058318e+07,36986.727368199536,8322,8322,50526.0,1619.3024491778801,266.7109009630352
1,2023-09-16 00:00:00+00:00,BSC,105265.85424866767,<nil>,1193,1193,5102.0,88.23625670466696,20.63227249091879
2,2023-09-16 00:00:00+00:00,Arbitrum,668.8733134123446,<nil>,3,3,3.0,222.9577711374482,222.9577711374482
3,2023-09-16 00:00:00+00:00,Solana,292.982798699,1.75426410688,1,1,11.0,292.982798699,26.63479988172727
4,2023-09-17 00:00:00+00:00,Ethereum,1.1929342885034062e+07,37326.3504461101,7526,3014,44517.0,1585.0840931482942,267.9727493998711


In [80]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

# Convert block_date to datetime if it's not already
df['block_date'] = pd.to_datetime(df['block_date'])

# Sort by date for proper plotting
df_sorted = df.sort_values('block_date')

print("Data columns:", df.columns.tolist())
print("\nUnique blockchains:", df['blockchain'].unique())
print(f"\nDate range: {df['block_date'].min()} to {df['block_date'].max()}")

Data columns: ['block_date', 'blockchain', 'volumeUSD', 'botRevenueUSD', 'numberOfUsers', 'numberOfNewUsers', 'numberOfTrades', 'averageVolumePerUserUSD', 'averageVolumePerTradeUSD']

Unique blockchains: ['Ethereum' 'BSC' 'Arbitrum' 'Solana' 'Base' 'Avalanche' 'Polygon'
 'Optimism' 'Fantom' 'Linea' 'Blast' 'Scroll' 'TON' 'sonic']

Date range: 2023-09-16 00:00:00+00:00 to 2025-09-10 00:00:00+00:00


In [102]:
# Create interactive plots with Plotly
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Volume USD by Chain', 'Number of Users by Chain', 
                   'Bot Revenue USD by Chain', 'Number of Trades by Chain'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Get unique blockchains and generate enough colors
blockchains = df['blockchain'].unique()
# Generate colors dynamically - cycle through color palettes if needed
colors = px.colors.qualitative.Set1 + px.colors.qualitative.Set2 + px.colors.qualitative.Set3
colors = colors[:len(blockchains)]  # Take only as many as needed

print(f"Number of blockchains: {len(blockchains)}")
print(f"Blockchains: {list(blockchains)}")

# Plot 1: Volume USD by Chain
for i, blockchain in enumerate(blockchains):
    chain_data = df_sorted[df_sorted['blockchain'] == blockchain]
    volume_col = pd.to_numeric(chain_data['volumeUSD'], errors='coerce')
    fig.add_trace(
        go.Scatter(x=chain_data['block_date'], y=volume_col,
                  name=f'{blockchain}', line=dict(color=colors[i]),
                  legendgroup=blockchain, showlegend=True),
        row=1, col=1
    )

# Plot 2: Number of Users by Chain  
for i, blockchain in enumerate(blockchains):
    chain_data = df_sorted[df_sorted['blockchain'] == blockchain]
    fig.add_trace(
        go.Scatter(x=chain_data['block_date'], y=chain_data['numberOfUsers'],
                  name=f'{blockchain}', line=dict(color=colors[i]),
                  legendgroup=blockchain, showlegend=False),
        row=1, col=2
    )

# Plot 3: Bot Revenue USD by Chain
for i, blockchain in enumerate(blockchains):
    chain_data = df_sorted[df_sorted['blockchain'] == blockchain]
    bot_revenue_col = pd.to_numeric(chain_data['botRevenueUSD'], errors='coerce')
    fig.add_trace(
        go.Scatter(x=chain_data['block_date'], y=bot_revenue_col,
                  name=f'{blockchain}', line=dict(color=colors[i]),
                  legendgroup=blockchain, showlegend=False),
        row=2, col=1
    )

# Plot 4: Number of Trades by Chain
for i, blockchain in enumerate(blockchains):
    chain_data = df_sorted[df_sorted['blockchain'] == blockchain]
    fig.add_trace(
        go.Scatter(x=chain_data['block_date'], y=chain_data['numberOfTrades'],
                  name=f'{blockchain}', line=dict(color=colors[i]),
                  legendgroup=blockchain, showlegend=False),
        row=2, col=2
    )

fig.update_layout(height=800, title_text="Trading Bot Activity by Blockchain Over Time")
fig.show()

Number of blockchains: 14
Blockchains: ['Ethereum', 'BSC', 'Arbitrum', 'Solana', 'Base', 'Avalanche', 'Polygon', 'Optimism', 'Fantom', 'Linea', 'Blast', 'Scroll', 'TON', 'sonic']


In [99]:
# Stacked area chart for all blockchains volume
fig = go.Figure()

# First, convert all data to numeric and calculate total volumes per blockchain
df_numeric = df.copy()
df_numeric['volumeUSD'] = pd.to_numeric(df_numeric['volumeUSD'], errors='coerce')

# Calculate total volume per blockchain to order them
blockchain_totals = df_numeric.groupby('blockchain')['volumeUSD'].sum().sort_values(ascending=False)
blockchains_ordered = blockchain_totals.index.tolist()  # Highest to lowest

# Define specific colors for key blockchains
blockchain_colors = {
    'Solana': 'rgba(138, 43, 226, 0.8)',      # Purple for Solana
    'Ethereum': 'rgba(54, 162, 235, 0.8)',    # Blue for Ethereum  
    'BSC': 'rgba(255, 205, 86, 0.8)',         # Yellow for BSC
    'Base': 'rgba(75, 192, 192, 0.8)',        # Teal for Base
    'TON': 'rgba(255, 99, 132, 0.8)',         # Red for TON
    'Avalanche': 'rgba(153, 102, 255, 0.8)',  # Light Purple
    'Arbitrum': 'rgba(255, 159, 64, 0.8)',    # Orange
    'sonic': 'rgba(199, 199, 199, 0.8)',      # Gray
    'Blast': 'rgba(83, 102, 255, 0.8)',       # Light Blue
    'Fantom': 'rgba(255, 99, 255, 0.8)',      # Pink
    'Polygon': 'rgba(99, 255, 132, 0.8)',     # Light Green
    'Optimism': 'rgba(255, 132, 99, 0.8)',    # Light Red
    'Linea': 'rgba(132, 255, 99, 0.8)',       # Green
    'Scroll': 'rgba(255, 165, 0, 0.8)'        # Dark Orange
}

print(f"Found {len(blockchains_ordered)} blockchains ordered by total volume:")
for i, blockchain in enumerate(blockchains_ordered):
    total_vol = blockchain_totals[blockchain]
    print(f"{i+1}. {blockchain}: ${total_vol:,.0f}")

# Add each blockchain in order from largest to smallest (largest becomes bottom layer)
for i, blockchain in enumerate(blockchains_ordered):
    blockchain_data = df_sorted[df_sorted['blockchain'] == blockchain].copy()
    
    if len(blockchain_data) > 0:
        # Ensure proper date sorting
        blockchain_data = blockchain_data.sort_values('block_date').reset_index(drop=True)
        
        # Convert volume to numeric (it's stored as string in scientific notation)
        blockchain_data['volumeUSD'] = pd.to_numeric(blockchain_data['volumeUSD'], errors='coerce')
        
        # Remove any NaN values that might have been created
        blockchain_data = blockchain_data.dropna(subset=['volumeUSD'])
        
        # Get color for this blockchain
        color = blockchain_colors.get(blockchain, f'rgba({50 + i*30}, {100 + i*20}, {150 + i*10}, 0.7)')
        
        fig.add_trace(
            go.Scatter(
                x=blockchain_data['block_date'], 
                y=blockchain_data['volumeUSD'],
                name=blockchain,
                fill='tonexty' if i > 0 else 'tozeroy',  # First one fills to zero, others stack on top
                mode='none',  # No lines, just filled area
                fillcolor=color,
                hovertemplate=f'<b>{blockchain}</b><br>Date: %{{x}}<br>Volume: $%{{y:,.0f}}<extra></extra>',
                stackgroup='one'  # Enable stacking
            )
        )

# Clean formatting
fig.update_layout(
    title='Bot Volume Last 765 Days', 
    title_x=0.5,
    xaxis_title='Date', 
    yaxis_title='Volume USD', 
    height=600,
    width=1200,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family="Arial", size=12),
    showlegend=True,
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02
    ),
    yaxis=dict(
        tickformat='$,.0f',
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=1
    ),
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=1
    )
)

fig.show()

Found 14 blockchains ordered by total volume:
1. Solana: $60,317,064,018
2. Ethereum: $17,279,944,601
3. Base: $5,139,507,580
4. BSC: $2,623,288,108
5. TON: $178,426,540
6. Avalanche: $48,412,365
7. Arbitrum: $21,546,576
8. sonic: $4,980,486
9. Blast: $992,765
10. Fantom: $123,868
11. Polygon: $39,571
12. Optimism: $24,829
13. Linea: $3,650
14. Scroll: $8


In [100]:
# Summary Statistics and Trends
print("=== TRADING BOT ACTIVITY SUMMARY ===\n")

# First, convert all numeric columns from scientific notation strings to proper numbers
df_numeric = df.copy()
numeric_columns = ['volumeUSD', 'botRevenueUSD', 'numberOfUsers', 'numberOfTrades', 'numberOfNewUsers', 'averageVolumePerUserUSD', 'averageVolumePerTradeUSD']

for col in numeric_columns:
    if col in df_numeric.columns:
        df_numeric[col] = pd.to_numeric(df_numeric[col], errors='coerce')

# Overall statistics
total_volume = df_numeric['volumeUSD'].sum()
total_revenue = df_numeric['botRevenueUSD'].sum()
total_trades = df_numeric['numberOfTrades'].sum()
avg_users_per_day = df_numeric['numberOfUsers'].mean()

print(f"Total Volume: ${total_volume:,.2f}")
print(f"Total Bot Revenue: ${total_revenue:,.2f}")
print(f"Total Trades: {total_trades:,}")
print(f"Average Users per Day: {avg_users_per_day:.0f}")
print(f"Revenue as % of Volume: {(total_revenue/total_volume)*100:.2f}%\n")

# By Blockchain
print("=== BY BLOCKCHAIN ===")
blockchain_summary = df_numeric.groupby('blockchain').agg({
    'volumeUSD': ['sum', 'mean'],
    'botRevenueUSD': ['sum', 'mean'],
    'numberOfUsers': ['sum', 'mean'],
    'numberOfTrades': ['sum', 'mean'],
    'numberOfNewUsers': 'sum'
}).round(2)

blockchain_summary.columns = ['Total_Volume', 'Avg_Daily_Volume', 'Total_Revenue', 'Avg_Daily_Revenue',
                             'Total_Users', 'Avg_Daily_Users', 'Total_Trades', 'Avg_Daily_Trades', 'Total_New_Users']

print(blockchain_summary)

# Recent trends (last 30 days)
recent_data = df_numeric[df_numeric['block_date'] >= df_numeric['block_date'].max() - pd.Timedelta(days=30)]
print(f"\n=== RECENT TRENDS (Last 30 Days) ===")
print(f"Recent Volume: ${recent_data['volumeUSD'].sum():,.2f}")
print(f"Recent Revenue: ${recent_data['botRevenueUSD'].sum():,.2f}")
print(f"Most Active Chain: {recent_data.groupby('blockchain')['volumeUSD'].sum().idxmax()}")

=== TRADING BOT ACTIVITY SUMMARY ===

Total Volume: $85,614,354,965.39
Total Bot Revenue: $550,747,964.64
Total Trades: 425,606,301.0
Average Users per Day: 8776
Revenue as % of Volume: 0.64%

=== BY BLOCKCHAIN ===
            Total_Volume  Avg_Daily_Volume  Total_Revenue  Avg_Daily_Revenue  \
blockchain                                                                     
Arbitrum    2.154658e+07          57304.72   4.929000e+01               1.49   
Avalanche   4.841237e+07          77708.45   4.652735e+04             124.74   
BSC         2.623288e+09        3613344.50   7.248436e+06           11134.31   
Base        5.139508e+09        7882680.34   1.794668e+07           27525.58   
Blast       9.927655e+05           7464.40   7.619600e+02              15.87   
Ethereum    1.727994e+10       23801576.59   5.360204e+07           73832.01   
Fantom      1.238681e+05           1492.39  -3.270220e+03             -42.47   
Linea       3.649560e+03            608.26   0.000000e+00        

Simple Sentiment Analysis Using Vader for Tweets

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    scores = analyzer.polarity_scores(text)
    return scores['compound']

sample_tweet = "Bitcoin is going to the moon!"
print(f"Sentiment score: {analyze_sentiment(sample_tweet)}")


Training a Random Forest Classifier on Features

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report

# Assuming df_features with feature columns and 'target' label
X = df_features.drop('target', axis=1)
y = df_features['target']

tscv = TimeSeriesSplit(n_splits=5)
model = RandomForestClassifier(n_estimators=100)

for train_idx, test_idx in tscv.split(X):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
